In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("ALPHA_APIKEY")

## IBM Stock prices

In [3]:
import requests

url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=ALPHA_APIKEY"
response = requests.get(url)
print(response.status_code)
data = response.json()
print(data)


200
{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'IBM', '3. Last Refreshed': '2025-06-02', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-06-02': {'1. open': '257.8500', '2. high': '263.9760', '3. low': '257.2200', '4. close': '263.9000', '5. volume': '2831881'}, '2025-05-30': {'1. open': '258.7500', '2. high': '260.1200', '3. low': '257.1000', '4. close': '259.0600', '5. volume': '9668923'}, '2025-05-29': {'1. open': '260.7500', '2. high': '261.1300', '3. low': '256.7700', '4. close': '258.6900', '5. volume': '2295228'}, '2025-05-28': {'1. open': '263.1600', '2. high': '265.0000', '3. low': '259.9400', '4. close': '260.2400', '5. volume': '2318437'}, '2025-05-27': {'1. open': '261.0000', '2. high': '263.7869', '3. low': '259.6300', '4. close': '263.2300', '5. volume': '3284216'}, '2025-05-23': {'1. open': '258.5800', '2. high': '259.8696', '3. low': '255.7900', '4. close': '258.6300', '5. 

In [4]:
print("Time Series (Daily):")
for date, stats in data["Time Series (Daily)"].items():
    print(f"Date: {date}")
    print(f"Open: {stats['1. open']}")
    print(f"High: {stats['2. high']}")
    print(f"Low: {stats['3. low']}")
    print(f"Close: {stats['4. close']}")
    print(f"Volume: {stats['5. volume']}")
    print()

Time Series (Daily):
Date: 2025-06-02
Open: 257.8500
High: 263.9760
Low: 257.2200
Close: 263.9000
Volume: 2831881

Date: 2025-05-30
Open: 258.7500
High: 260.1200
Low: 257.1000
Close: 259.0600
Volume: 9668923

Date: 2025-05-29
Open: 260.7500
High: 261.1300
Low: 256.7700
Close: 258.6900
Volume: 2295228

Date: 2025-05-28
Open: 263.1600
High: 265.0000
Low: 259.9400
Close: 260.2400
Volume: 2318437

Date: 2025-05-27
Open: 261.0000
High: 263.7869
Low: 259.6300
Close: 263.2300
Volume: 3284216

Date: 2025-05-23
Open: 258.5800
High: 259.8696
Low: 255.7900
Close: 258.6300
Volume: 2722721

Date: 2025-05-22
Open: 260.7700
High: 261.2711
Low: 257.9100
Close: 258.3700
Volume: 3091253

Date: 2025-05-21
Open: 264.9700
High: 265.6499
Low: 260.4100
Close: 260.8700
Volume: 3753904

Date: 2025-05-20
Open: 267.4000
High: 269.2800
Low: 265.6201
Close: 266.9500
Volume: 2437860

Date: 2025-05-19
Open: 265.4500
High: 269.1350
Low: 265.0800
Close: 268.4100
Volume: 3198903

Date: 2025-05-16
Open: 266.3500
High: 2

In [5]:
import pandas as pd
import sqlite3

# Example: Parse your API data into a list of dicts
records = []
symbol = "IBM"
for date, stats in data["Time Series (Daily)"].items():
    records.append({
        "symbol": symbol,
        "date": date,
        "open": float(stats["1. open"]),
        "high": float(stats["2. high"]),
        "low": float(stats["3. low"]),
        "close": float(stats["4. close"]),
        "volume": int(stats["5. volume"])
    })

stock_df = pd.DataFrame(records)

# Insert into SQLite
conn = sqlite3.connect("stock_prices.db")
stock_df.to_sql("daily_stock_prices", conn, if_exists="append", index=False)
conn.commit()
conn.close()

IntegrityError: UNIQUE constraint failed: daily_stock_prices.symbol, daily_stock_prices.date

In [6]:
import sqlite3

# Connect to the database (it will be created if it doesn't exist)
conn = sqlite3.connect("stock_prices.db")
cursor = conn.cursor()

# Create the table
cursor.execute("""
CREATE TABLE IF NOT EXISTS daily_stock_prices (
    symbol TEXT NOT NULL,
    date TEXT NOT NULL,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume INTEGER,
    PRIMARY KEY (symbol, date)
);
""")

conn.commit()
conn.close()

In [7]:
import pandas as pd
# Reconnect to the database because conn was closed previously
import sqlite3

conn = sqlite3.connect("stock_prices.db")
df_all = pd.read_sql_query("SELECT * FROM daily_stock_prices", conn)
print(df_all)
conn.close()

   symbol        date    open      high       low   close   volume
0     IBM  2025-05-23  258.58  259.8696  255.7900  258.63  2722721
1     IBM  2025-05-22  260.77  261.2711  257.9100  258.37  3091253
2     IBM  2025-05-21  264.97  265.6499  260.4100  260.87  3753904
3     IBM  2025-05-20  267.40  269.2800  265.6201  266.95  2437860
4     IBM  2025-05-19  265.45  269.1350  265.0800  268.41  3198903
..    ...         ...     ...       ...       ...     ...      ...
95    IBM  2025-01-06  223.00  224.3500  220.7500  222.67  2847128
96    IBM  2025-01-03  220.55  223.6600  220.5500  222.65  3873578
97    IBM  2025-01-02  221.82  222.4900  217.6000  219.94  2579498
98    IBM  2024-12-31  220.72  221.0493  218.4400  219.83  2270512
99    IBM  2024-12-30  220.54  221.5942  217.6523  220.25  2095565

[100 rows x 7 columns]


In [8]:
import pandas as pd
import sqlite3

# Example: Parse your API data into a list of dicts
records = []
symbol = "IBM"
for date, stats in data["Time Series (Daily)"].items():
    records.append({
        "symbol": symbol,
        "date": date,
        "open": float(stats["1. open"]),
        "high": float(stats["2. high"]),
        "low": float(stats["3. low"]),
        "close": float(stats["4. close"]),
        "volume": int(stats["5. volume"])
    })

stock_df = pd.DataFrame(records)

# Insert into SQLite
conn = sqlite3.connect("stock_prices.db")
stock_df.to_sql("daily_stock_prices", conn, if_exists="append", index=False)
conn.commit()
conn.close()

IntegrityError: UNIQUE constraint failed: daily_stock_prices.symbol, daily_stock_prices.date

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("stock_prices.db")
df_all = pd.read_sql_query("SELECT * FROM daily_stock_prices", conn)
print(df_all)
conn.close()

   symbol        date    open      high       low   close   volume
0     IBM  2025-05-23  258.58  259.8696  255.7900  258.63  2722721
1     IBM  2025-05-22  260.77  261.2711  257.9100  258.37  3091253
2     IBM  2025-05-21  264.97  265.6499  260.4100  260.87  3753904
3     IBM  2025-05-20  267.40  269.2800  265.6201  266.95  2437860
4     IBM  2025-05-19  265.45  269.1350  265.0800  268.41  3198903
..    ...         ...     ...       ...       ...     ...      ...
95    IBM  2025-01-06  223.00  224.3500  220.7500  222.67  2847128
96    IBM  2025-01-03  220.55  223.6600  220.5500  222.65  3873578
97    IBM  2025-01-02  221.82  222.4900  217.6000  219.94  2579498
98    IBM  2024-12-31  220.72  221.0493  218.4400  219.83  2270512
99    IBM  2024-12-30  220.54  221.5942  217.6523  220.25  2095565

[100 rows x 7 columns]


In [ ]:
stock_df.to_csv("stock_prices.csv", index=False)

## Bitcoin Stock Prices

In [10]:
import requests

url = "https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=EUR&apikey=ALPHA_APIKEY"
response = requests.get(url)
print(response.status_code)
data_btc = response.json()
print(data_btc)


200
{'Meta Data': {'1. Information': 'Daily Prices and Volumes for Digital Currency', '2. Digital Currency Code': 'BTC', '3. Digital Currency Name': 'Bitcoin', '4. Market Code': 'EUR', '5. Market Name': 'Euro', '6. Last Refreshed': '2025-06-03 00:00:00', '7. Time Zone': 'UTC'}, 'Time Series (Digital Currency Daily)': {'2025-06-03': {'1. open': '92492.40000000', '2. high': '93037.19000000', '3. low': '92403.27000000', '4. close': '92911.79000000', '5. volume': '3.60335916'}, '2025-06-02': {'1. open': '93129.28000000', '2. high': '93133.86000000', '3. low': '90726.84000000', '4. close': '92494.38000000', '5. volume': '221.00441388'}, '2025-06-01': {'1. open': '92308.49000000', '2. high': '93307.12000000', '3. low': '91577.88000000', '4. close': '93142.55000000', '5. volume': '95.29867667'}, '2025-05-31': {'1. open': '91712.80000000', '2. high': '92599.72000000', '3. low': '90942.82000000', '4. close': '92311.30000000', '5. volume': '122.89772071'}, '2025-05-30': {'1. open': '92825.400000

In [11]:
print(data_btc.keys())

dict_keys(['Meta Data', 'Time Series (Digital Currency Daily)'])


In [12]:
print("Time Series (Digital Currency Daily):")
for date, stats in data_btc["Time Series (Digital Currency Daily)"].items():
    print(f"Date: {date}")
    print(f"Open: {stats['1. open']}")
    print(f"High: {stats['2. high']}")
    print(f"Low: {stats['3. low']}")
    print(f"Close: {stats['4. close']}")
    print(f"Volume: {stats['5. volume']}")
    print()

Time Series (Digital Currency Daily):
Date: 2025-06-03
Open: 92492.40000000
High: 93037.19000000
Low: 92403.27000000
Close: 92911.79000000
Volume: 3.60335916

Date: 2025-06-02
Open: 93129.28000000
High: 93133.86000000
Low: 90726.84000000
Close: 92494.38000000
Volume: 221.00441388

Date: 2025-06-01
Open: 92308.49000000
High: 93307.12000000
Low: 91577.88000000
Close: 93142.55000000
Volume: 95.29867667

Date: 2025-05-31
Open: 91712.80000000
High: 92599.72000000
Low: 90942.82000000
Close: 92311.30000000
Volume: 122.89772071

Date: 2025-05-30
Open: 92825.40000000
High: 93749.99000000
Low: 91369.44000000
Close: 91725.02000000
Volume: 287.20854273

Date: 2025-05-29
Open: 95949.10000000
High: 96548.03000000
Low: 92580.75000000
Close: 92822.04000000
Volume: 303.13067402

Date: 2025-05-28
Open: 96030.74000000
High: 96570.99000000
Low: 94577.00000000
Close: 95943.76000000
Volume: 201.78380507

Date: 2025-05-27
Open: 96109.72000000
High: 97493.63000000
Low: 94300.00000000
Close: 96102.83000000
Vol

In [13]:
import pandas as pd
import sqlite3

# Example: Parse your API data into a list of dicts
records = []
symbol = "BTC"
for date, stats in data_btc["Time Series (Digital Currency Daily)"].items():
    records.append({
        "symbol": symbol,
        "date": date,
        "open": float(stats["1. open"]),
        "high": float(stats["2. high"]),
        "low": float(stats["3. low"]),
        "close": float(stats["4. close"]),
        "volume": float(stats["5. volume"])
    })

stock_df = pd.DataFrame(records)

# Insert into SQLite
conn = sqlite3.connect("stock_prices_btc.db")
stock_df.to_sql("digital_currency_daily_btc", conn, if_exists="append", index=False)
conn.commit()
conn.close()

In [14]:
import sqlite3

# Connect to the database (it will be created if it doesn't exist)
conn = sqlite3.connect("stock_prices_btc.db")
cursor = conn.cursor()

# Create the table
cursor.execute("""
CREATE TABLE IF NOT EXISTS digital_currency_daily_btc (
    symbol TEXT NOT NULL,
    date TEXT NOT NULL,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume REAL,
    PRIMARY KEY (symbol, date)
);
""")

conn.commit()
conn.close()

In [15]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("stock_prices_btc.db")
df_all = pd.read_sql_query("SELECT * FROM digital_currency_daily_btc", conn)
print(df_all)
conn.close()

    symbol        date      open      high       low     close      volume
0      BTC  2025-05-27  96109.72  96255.56  95781.83  95804.40    2.104122
1      BTC  2025-05-26  95885.75  97065.11  95460.32  96120.94  190.813164
2      BTC  2025-05-25  94896.77  96151.29  93901.00  95889.39  146.060603
3      BTC  2025-05-24  94433.13  96379.79  94106.49  94894.79  122.420715
4      BTC  2025-05-23  98907.04  98965.34  94001.00  94483.29  384.613598
..     ...         ...       ...       ...       ...       ...         ...
695    BTC  2024-06-23  59996.32  60246.86  59085.16  59107.12   89.241386
696    BTC  2024-06-22  59995.39  60336.51  59797.74  59992.10   74.278082
697    BTC  2024-06-21  60577.71  60719.65  59287.61  59987.59  270.544731
698    BTC  2024-06-20  60466.69  61937.39  60249.86  60550.86  323.100504
699    BTC  2024-06-19  60698.63  61218.74  60200.01  60456.22  168.566977

[700 rows x 7 columns]


In [16]:
stock_df.to_csv("btc_prices.csv", index=False)